<a href="https://colab.research.google.com/github/FrostBt/Seq-IDs/blob/main/script1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse

def read_fasta(filename):
    sequences = []
    current_seq = None
    with open(filename, "r") as fasta_file:
        for line in fasta_file:
            line = line.strip()
            if line.startswith(">"):
                if current_seq:
                    sequences.append(current_seq)
                current_seq = {"ID": line[1:], "Sequência": "", "Descrição": ""}
            else:
                current_seq["Sequência"] += line
        if current_seq:
            sequences.append(current_seq)
    return sequences

def count_nucleotides(sequence):
    count_a = sequence.count("A")
    count_t = sequence.count("T")
    count_c = sequence.count("C")
    count_g = sequence.count("G")
    return count_a, count_t, count_c, count_g

def main(args):
    if args.input_file:
        sequences = read_fasta(args.input_file)
    else:
        sequences = []
        for i in range(args.num_sequences):
            dna_seq = args.sequences[i]
            dna_id = args.ids[i]
            dna_desc = args.descriptions[i] if args.descriptions else ""
            sequences.append({"ID": dna_id, "Sequência": dna_seq, "Descrição": dna_desc})

    for seq in sequences:
        if not seq["ID"]:
            raise ValueError("O ID não pode estar em branco")
        if set(seq["Sequência"]) - set("ATCG"):
            raise ValueError("A sequência de DNA deve conter apenas as letras 'A', 'T', 'C' e 'G'")

    for seq in sequences:
        count_a, count_t, count_c, count_g = count_nucleotides(seq["Sequência"])
        seq["Contagem"] = {"A": count_a, "T": count_t, "C": count_c, "G": count_g}

    if args.output_file:
        filename = args.output_file
    else:
        filename = "sequences.fasta"

    with open(filename, "w") as fasta_file:
        for seq in sequences:
            header = f">{seq['ID']}|{seq['Descrição']}|A:{seq['Contagem']['A']}|T:{seq['Contagem']['T']}|C:{seq['Contagem']['C']}|G:{seq['Contagem']['G']}"
            fasta_file.write(f"{header}\n{seq['Sequência']}\n")

    print(f"Sequências salvas no arquivo '{filename}'.")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Processa sequências de DNA e gera um arquivo FASTA.")
    parser.add_argument("--input-file", help="Arquivo FASTA de entrada")
    parser.add_argument("--output-file", help="Nome do arquivo FASTA de saída")
    parser.add_argument("--sequences", nargs="+", help="Sequências de DNA")
    parser.add_argument("--ids", nargs="+", help="IDs das sequências")
    parser.add_argument("--descriptions", nargs="+", help="Descrições das sequências")
    parser.add_argument("--num-sequences", type=int, default=1, help="Número total de sequências")
    args = parser.parse_args()
    main(args)

